In [1]:
import numpy as np
import pandas as pd

#scikit-learn class that helps to take care of missing values
from sklearn.impute import SimpleImputer

# to handle outliers
from scipy.stats import zscore

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# for standardizing the features
from sklearn.preprocessing import StandardScaler


#### Connecting the Colab Environment to the Google Drive  account.So that we can access and manipulate files stored in a Drive directly from Colab.

In [2]:
from google.colab import drive
drive.mount('companies.csv')

Mounted at companies.csv


#### Loading dataset

In [3]:
data = pd.read_csv('/content/companies.csv/MyDrive/companies.csv')

In [4]:
data.head(10)

,id,Unnamed: 0.1,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,...,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,lat,lng,ROI
0,c:1,0,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,...,2010-09-05,2013-09-18,5.0,17.0,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,47.606209,-122.332071,15.5
1,c:10,1,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,...,NaN,NaN,NaN,6.0,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,34.021122,-118.396467,NaN
2,c:100,2,Company,100,NaN,There,there,/company/there,games_video,acquired,...,2003-02-01,2011-09-23,4.0,12.0,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,37.562992,-122.325525,NaN
3,c:10000,3,Company,10000,NaN,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18,NaN,NaN,NaN
4,c:10001,4,Company,10001,NaN,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18,NaN,NaN,NaN
5,c:10002,5,Company,10002,NaN,Synergie Media,synergie media,/company/synergie-media,advertising,operating,...,NaN,NaN,NaN,2.0,NaN,2008-08-24 17:39:10,2008-09-06 14:19:19,30.427755,-9.598107,NaN
6,c:10003,6,Company,10003,NaN,Green Basti Ecosystems,green basti ecosystems,/company/green-basti-ecosystems,cleantech,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 19:49:25,2010-04-22 23:48:20,22.307159,73.181219,NaN
7,c:100042,7,Company,100042,NaN,HostNine,hostnine,/company/hostnine,NaN,operating,...,2011-06-20,2011-06-20,1.0,1.0,NaN,2011-09-07 00:53:17,2011-12-20 09:09:53,NaN,NaN,NaN
8,c:10005,8,Company,10005,NaN,Thomas Publishing,thomas publishing,/company/thomas-publishing,advertising,operating,...,1999-06-01,1999-06-01,1.0,2.0,NaN,2008-08-24 20:21:21,2009-11-19 17:21:00,40.712775,-74.005973,NaN
9,c:100062,9,Company,100062,NaN,Vetter Idea Management System,vetter idea management system,/company/vetter-idea-management-system,enterprise,operating,...,2011-08-01,2011-08-01,1.0,2.0,dmurtagh,2011-09-07 01:04:56,2011-09-24 13:13:18,NaN,NaN,NaN


#### Checking for Null and Duplicates

In [5]:
data.isnull().sum()

id                          0
Unnamed: 0.1                0
entity_type                 0
entity_id                   0
parent_id              196553
name                       22
normalized_name            26
permalink                   0
category_code           73367
status                      0
founded_at             105326
closed_at              193933
domain                  70008
homepage_url            70008
twitter_username       115962
logo_url                86443
logo_width              86443
logo_height             86443
short_description      189422
description            104505
overview                69582
tag_list               115101
country_code           108563
state_code             145650
city                   112663
region                      0
first_investment_at    193970
last_investment_at     193970
investment_rounds      193962
invested_companies     193962
first_funding_at       165046
last_funding_at        165046
funding_rounds         164846
funding_to

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196553 entries, 0 to 196552
Data columns (total 44 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   196553 non-null  object 
 1   Unnamed: 0.1         196553 non-null  int64  
 2   entity_type          196553 non-null  object 
 3   entity_id            196553 non-null  int64  
 4   parent_id            0 non-null       float64
 5   name                 196531 non-null  object 
 6   normalized_name      196527 non-null  object 
 7   permalink            196553 non-null  object 
 8   category_code        123186 non-null  object 
 9   status               196553 non-null  object 
 10  founded_at           91227 non-null   object 
 11  closed_at            2620 non-null    object 
 12  domain               126545 non-null  object 
 13  homepage_url         126545 non-null  object 
 14  twitter_username     80591 non-null   object 
 15  logo_url         

In [7]:
data.describe()

,Unnamed: 0.1,entity_id,parent_id,logo_width,logo_height,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,lat,lng,ROI
count,196553.000000,196553.000000,0.0,110110.000000,110110.000000,2591.000000,2591.000000,31707.000000,2.787400e+04,91699.000000,129667.000000,83852.000000,83852.000000,726.000000
mean,98276.000000,153006.227333,NaN,459.132685,222.728917,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,37.564512,-52.123066,45.745037
std,56740.108067,90209.250941,NaN,594.982577,333.090722,12.173510,11.436955,1.201666,6.775937e+07,0.540099,9.100309,15.477102,70.049067,572.035638
min,0.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,2.910000e+02,1.000000,1.000000,-50.942326,-159.497746,0.011111
25%,49138.000000,59850.000000,NaN,192.000000,70.000000,1.000000,1.000000,1.000000,5.000000e+05,1.000000,1.000000,34.052234,-111.940005,2.648879
50%,98276.000000,174539.000000,NaN,267.000000,105.000000,1.000000,1.000000,1.000000,2.564500e+06,1.000000,1.000000,39.768403,-77.036871,6.500497
75%,147414.000000,232655.000000,NaN,484.000000,232.000000,1.000000,1.000000,2.000000,1.100000e+07,1.000000,3.000000,45.421530,-0.127758,13.549900
max,196552.000000,286215.000000,NaN,18200.000000,9600.000000,478.000000,459.000000,15.000000,5.700000e+09,9.000000,1189.000000,77.553604,176.212549,13333.333333


#### Calculating percentage of nulls

In [8]:
data.isnull().mean()* 100

id                       0.000000
Unnamed: 0.1             0.000000
entity_type              0.000000
entity_id                0.000000
parent_id              100.000000
name                     0.011193
normalized_name          0.013228
permalink                0.000000
category_code           37.326828
status                   0.000000
founded_at              53.586564
closed_at               98.667026
domain                  35.617874
homepage_url            35.617874
twitter_username        58.997828
logo_url                43.979486
logo_width              43.979486
logo_height             43.979486
short_description       96.371971
description             53.168865
overview                35.401139
tag_list                58.559778
country_code            55.233448
state_code              74.102151
city                    57.319400
region                   0.000000
first_investment_at     98.685851
last_investment_at      98.685851
investment_rounds       98.681780
invested_compa

In [9]:
data.shape[1]

44

In [10]:
data.duplicated().any()

False

In [11]:
# dropping multiple columns
columns_to_drop = ['entity_type','parent_id', 'name', 'permalink', 'domain', 'twitter_username', 'logo_url', 'logo_width', 'logo_height', 'short_description', 'description', 'overview', 'tag_list', 'lat', 'lng']

In [12]:
data = data.drop(columns = columns_to_drop, axis=1)

In [13]:
data.head()

,id,Unnamed: 0.1,entity_id,normalized_name,category_code,status,founded_at,closed_at,homepage_url,country_code,...,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,ROI
0,c:1,0,1,wetpaint,web,operating,2005-10-17,NaN,http://wetpaint-inc.com,USA,...,3.0,39750000.0,2010-09-05,2013-09-18,5.0,17.0,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,15.5
1,c:10,1,10,flektor,games_video,acquired,NaN,NaN,http://www.flektor.com,USA,...,NaN,NaN,NaN,NaN,NaN,6.0,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,NaN
2,c:100,2,100,there,games_video,acquired,NaN,NaN,http://www.there.com,USA,...,NaN,NaN,2003-02-01,2011-09-23,4.0,12.0,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,NaN
3,c:10000,3,10000,mywebbo,network_hosting,operating,2008-07-26,NaN,http://www.mywebbo.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18,NaN
4,c:10001,4,10001,the movie streamer,games_video,operating,2008-07-26,NaN,http://themoviestreamer.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18,NaN


In [14]:
data.describe()

,Unnamed: 0.1,entity_id,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,ROI
count,196553.000000,196553.000000,2591.000000,2591.000000,31707.000000,2.787400e+04,91699.000000,129667.000000,726.000000
mean,98276.000000,153006.227333,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,45.745037
std,56740.108067,90209.250941,12.173510,11.436955,1.201666,6.775937e+07,0.540099,9.100309,572.035638
min,0.000000,1.000000,1.000000,1.000000,1.000000,2.910000e+02,1.000000,1.000000,0.011111
25%,49138.000000,59850.000000,1.000000,1.000000,1.000000,5.000000e+05,1.000000,1.000000,2.648879
50%,98276.000000,174539.000000,1.000000,1.000000,1.000000,2.564500e+06,1.000000,1.000000,6.500497
75%,147414.000000,232655.000000,1.000000,1.000000,2.000000,1.100000e+07,1.000000,3.000000,13.549900
max,196552.000000,286215.000000,478.000000,459.000000,15.000000,5.700000e+09,9.000000,1189.000000,13333.333333


In [15]:
data.isnull().sum()

id                          0
Unnamed: 0.1                0
entity_id                   0
normalized_name            26
category_code           73367
status                      0
founded_at             105326
closed_at              193933
homepage_url            70008
country_code           108563
state_code             145650
city                   112663
region                      0
first_investment_at    193970
last_investment_at     193970
investment_rounds      193962
invested_companies     193962
first_funding_at       165046
last_funding_at        165046
funding_rounds         164846
funding_total_usd      168679
first_milestone_at     104854
last_milestone_at      104854
milestones             104854
relationships           66886
created_by              41020
created_at                  0
updated_at                  0
ROI                    195827
dtype: int64

In [16]:
data = data.drop(columns ='closed_at', axis=1)

In [17]:
data = data.drop(columns ='homepage_url', axis=1)

### Data Cleaning


1.   Handling missing values using mean imputation for numeric datas.
2.   Handling missing values using mode imputation for categorical datas.
3.   Handling Outliers


In [18]:
# handling the numeric missing values
numeric_columns = ['investment_rounds','invested_companies','funding_rounds', 'funding_total_usd', 'milestones', 'relationships','ROI']
categorical_columns = ['normalized_name', 'category_code', 'founded_at', 'country_code','state_code', 'city', 'first_investment_at', 'last_investment_at','first_funding_at', 'last_funding_at', 'first_milestone_at','last_milestone_at','created_by']

In [19]:
#imputing missing values in numeric columns using mean
numeric_imputer = SimpleImputer(strategy='mean')
data[numeric_columns] = numeric_imputer.fit_transform(data[numeric_columns])

In [20]:
# imputing missing values in categorical columns using mode
categorical_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_columns] = categorical_imputer.fit_transform(data[categorical_columns])

In [21]:
data.isnull().sum()

id                     0
Unnamed: 0.1           0
entity_id              0
normalized_name        0
category_code          0
status                 0
founded_at             0
country_code           0
state_code             0
city                   0
region                 0
first_investment_at    0
last_investment_at     0
investment_rounds      0
invested_companies     0
first_funding_at       0
last_funding_at        0
funding_rounds         0
funding_total_usd      0
first_milestone_at     0
last_milestone_at      0
milestones             0
relationships          0
created_by             0
created_at             0
updated_at             0
ROI                    0
dtype: int64

In [22]:
data.describe()

,Unnamed: 0.1,entity_id,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,ROI
count,196553.000000,196553.000000,196553.000000,196553.000000,196553.000000,1.965530e+05,196553.000000,196553.000000,196553.000000
mean,98276.000000,153006.227333,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,45.745037
std,56740.108067,90209.250941,1.397420,1.312869,0.482632,2.551658e+07,0.368905,7.391464,34.741883
min,0.000000,1.000000,1.000000,1.000000,1.000000,2.910000e+02,1.000000,1.000000,0.011111
25%,49138.000000,59850.000000,2.372057,2.204940,1.659760,1.481652e+07,1.000000,1.000000,45.745037
50%,98276.000000,174539.000000,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,45.745037
75%,147414.000000,232655.000000,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,45.745037
max,196552.000000,286215.000000,478.000000,459.000000,15.000000,5.700000e+09,9.000000,1189.000000,13333.333333


In [23]:
#imputing missing values in numeric columns using median
numeric_imputer = SimpleImputer(strategy='median')
data[numeric_columns] = numeric_imputer.fit_transform(data[numeric_columns])

In [24]:
data.describe()

,Unnamed: 0.1,entity_id,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,ROI
count,196553.000000,196553.000000,196553.000000,196553.000000,196553.000000,1.965530e+05,196553.000000,196553.000000,196553.000000
mean,98276.000000,153006.227333,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,45.745037
std,56740.108067,90209.250941,1.397420,1.312869,0.482632,2.551658e+07,0.368905,7.391464,34.741883
min,0.000000,1.000000,1.000000,1.000000,1.000000,2.910000e+02,1.000000,1.000000,0.011111
25%,49138.000000,59850.000000,2.372057,2.204940,1.659760,1.481652e+07,1.000000,1.000000,45.745037
50%,98276.000000,174539.000000,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,45.745037
75%,147414.000000,232655.000000,2.372057,2.204940,1.659760,1.481652e+07,1.199402,2.852067,45.745037
max,196552.000000,286215.000000,478.000000,459.000000,15.000000,5.700000e+09,9.000000,1189.000000,13333.333333


#### Using Z-score to handle outliers

In [25]:
# calculating z-score for numeric columns
z_scores = np.abs(zscore(data[numeric_columns]))

In [26]:
z_scores

,investment_rounds,invested_companies,funding_rounds,funding_total_usd,milestones,relationships,ROI
0,3.177930e-16,3.382593e-16,2.776950,9.771507e-01,1.030240e+01,1.914096,8.705663e-01
1,3.177930e-16,3.382593e-16,0.000000,7.299763e-17,6.019035e-16,0.425889,4.090420e-16
2,3.177930e-16,3.382593e-16,0.000000,7.299763e-17,7.591671e+00,1.237638,4.090420e-16
3,3.177930e-16,3.382593e-16,0.000000,7.299763e-17,6.019035e-16,0.000000,4.090420e-16
4,3.177930e-16,3.382593e-16,0.000000,7.299763e-17,6.019035e-16,0.000000,4.090420e-16
...,...,...,...,...,...,...,...
196548,3.177930e-16,3.382593e-16,0.000000,7.299763e-17,2.170206e+00,0.290597,4.090420e-16
196549,3.177930e-16,3.382593e-16,1.367008,5.512712e-01,5.405265e-01,1.508221,4.090420e-16
196550,3.177930e-16,3.382593e-16,0.000000,7.299763e-17,4.880938e+00,5.566967,4.090420e-16
196551,3.177930e-16,3.382593e-16,0.000000,7.299763e-17,6.019035e-16,0.250569,4.090420e-16


In [27]:
# defining the threshold value for z-score (eg: 8 standard deviation)
threshold = 13.5

In [28]:
# identifying and removing rows with outliers
df_no_outliers = data[(z_scores < threshold).all(axis=1)]


In [29]:
data

,id,Unnamed: 0.1,entity_id,normalized_name,category_code,status,founded_at,country_code,state_code,city,...,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,ROI
0,c:1,0,1,wetpaint,web,operating,2005-10-17,USA,WA,Seattle,...,3.00000,3.975000e+07,2010-09-05,2013-09-18,5.000000,17.000000,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,15.500000
1,c:10,1,10,flektor,games_video,acquired,2011-01-01,USA,CA,Culver City,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,6.000000,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,45.745037
2,c:100,2,100,there,games_video,acquired,2011-01-01,USA,CA,San Mateo,...,1.65976,1.481652e+07,2003-02-01,2011-09-23,4.000000,12.000000,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,45.745037
3,c:10000,3,10000,mywebbo,network_hosting,operating,2008-07-26,USA,CA,New York,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,2.852067,dhawalarco,2008-08-24 16:51:57,2008-09-06 14:19:18,45.745037
4,c:10001,4,10001,the movie streamer,games_video,operating,2008-07-26,USA,CA,New York,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,2.852067,dhawalarco,2008-08-24 17:10:34,2008-09-06 14:19:18,45.745037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,196548,99940,webgility,ecommerce,operating,2007-01-01,USA,CA,San Francisco,...,1.65976,1.481652e+07,2013-08-12,2013-08-12,2.000000,5.000000,dhawalarco,2011-09-06 23:17:56,2013-08-13 00:39:13,45.745037
196549,c:9995,196549,9995,fohboh,public_relations,operating,2007-11-01,USA,CA,San Jose,...,1.00000,7.500000e+05,2013-05-01,2013-05-01,1.000000,14.000000,dhawalarco,2008-08-24 15:46:26,2013-06-28 20:55:30,45.745037
196550,c:9996,196550,9996,csc,consulting,operating,1959-01-01,USA,VA,Falls Church,...,1.65976,1.481652e+07,2012-01-01,2013-10-30,3.000000,44.000000,dhawalarco,2008-08-24 15:46:58,2013-06-28 20:52:01,45.745037
196551,c:9997,196551,9997,top candidate,search,operating,2008-07-01,USA,CA,Los Angeles,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,1.000000,dhawalarco,2008-08-24 16:00:22,2013-06-28 20:56:44,45.745037


In [30]:
df_no_outliers

,id,Unnamed: 0.1,entity_id,normalized_name,category_code,status,founded_at,country_code,state_code,city,...,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,ROI
0,c:1,0,1,wetpaint,web,operating,2005-10-17,USA,WA,Seattle,...,3.00000,3.975000e+07,2010-09-05,2013-09-18,5.000000,17.000000,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,15.500000
1,c:10,1,10,flektor,games_video,acquired,2011-01-01,USA,CA,Culver City,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,6.000000,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,45.745037
2,c:100,2,100,there,games_video,acquired,2011-01-01,USA,CA,San Mateo,...,1.65976,1.481652e+07,2003-02-01,2011-09-23,4.000000,12.000000,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,45.745037
3,c:10000,3,10000,mywebbo,network_hosting,operating,2008-07-26,USA,CA,New York,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,2.852067,dhawalarco,2008-08-24 16:51:57,2008-09-06 14:19:18,45.745037
4,c:10001,4,10001,the movie streamer,games_video,operating,2008-07-26,USA,CA,New York,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,2.852067,dhawalarco,2008-08-24 17:10:34,2008-09-06 14:19:18,45.745037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,196548,99940,webgility,ecommerce,operating,2007-01-01,USA,CA,San Francisco,...,1.65976,1.481652e+07,2013-08-12,2013-08-12,2.000000,5.000000,dhawalarco,2011-09-06 23:17:56,2013-08-13 00:39:13,45.745037
196549,c:9995,196549,9995,fohboh,public_relations,operating,2007-11-01,USA,CA,San Jose,...,1.00000,7.500000e+05,2013-05-01,2013-05-01,1.000000,14.000000,dhawalarco,2008-08-24 15:46:26,2013-06-28 20:55:30,45.745037
196550,c:9996,196550,9996,csc,consulting,operating,1959-01-01,USA,VA,Falls Church,...,1.65976,1.481652e+07,2012-01-01,2013-10-30,3.000000,44.000000,dhawalarco,2008-08-24 15:46:58,2013-06-28 20:52:01,45.745037
196551,c:9997,196551,9997,top candidate,search,operating,2008-07-01,USA,CA,Los Angeles,...,1.65976,1.481652e+07,2012-01-01,2012-01-01,1.199402,1.000000,dhawalarco,2008-08-24 16:00:22,2013-06-28 20:56:44,45.745037


#### Standardization of features helps to transform the data in such a way that has mean as 0 and Standard Deviation as 1.
The formula of standardization is given by

z= x - mean/ standard deviation


In [31]:
# scaling the numeric columns
scale = StandardScaler()
data[numeric_columns] = scale.fit_transform(data[numeric_columns])

In [32]:
mean_values = data[numeric_columns].mean()
std_values = data[numeric_columns].std()

print("Mean values:\n", mean_values)
print("\nStandard deviation values:\n", std_values)

Mean values:
 investment_rounds     3.187362e-16
invested_companies   -3.352749e-16
funding_rounds       -4.540463e-17
funding_total_usd    -6.702244e-17
milestones           -7.426694e-16
relationships        -5.205627e-18
ROI                  -4.093105e-16
dtype: float64

Standard deviation values:
 investment_rounds     1.000003
invested_companies    1.000003
funding_rounds        1.000003
funding_total_usd     1.000003
milestones            1.000003
relationships         1.000003
ROI                   1.000003
dtype: float64


#### Handling categorical values using OneHot Encoding

In [ ]:
# data_encoded = pd.get_dummies(data, columns=['normalized_name'])
# , 'category_code', 'founded_at', 'country_code','state_code', 'city', 'first_investment_at', 'last_investment_at','first_funding_at', 'last_funding_at', 'first_milestone_at','last_milestone_at','created_by'])